<a href="https://colab.research.google.com/github/callysthenes/reinforcement_learning/blob/main/constrained_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.optimize import minimize

In [ ]:
# Define the objective function
def objective(x):
    # note we are gonna minimize the objective function, but we wanna maximize revenue
    return -(10*x[0] + 20*x[1] + 30*x[2])

In [ ]:
# Define the constraint functions: 

# From scipy documentation:

# Equality constraint means that the constraint function result is to be zero 
# whereas inequality means that it is to be non-negative (>=0)

def capacity_constraint(x):
    # inequality
    return 25 - 2*x[0] - 3*x[1] - 4*x[2]

def demand_constraint(x):
    # equality
    return x[0] + x[1] + x[2] - 10

# Define the constraints
cons = ({'type': 'ineq', 'fun': capacity_constraint},
        {'type': 'eq', 'fun': demand_constraint})

In [ ]:
# Define the initial guess
x0 = np.array([1, 1, 1])

In [ ]:
# Define the bounds for the variables
bounds = ((0, None), (0, None), (0, None))

# Solve with SLSQP (Sequential Least Squares Programming )

In [ ]:
# Solve the optimization problem
result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)

# Print the results
print('Optimal solution:', result.x)
print('Optimal objective value:', -result.fun)

Optimal solution: [5.83333333 3.33333333 0.83333333]
Optimal objective value: 149.9999999999895


In [ ]:
x = result.x
-objective(x), capacity_constraint(x), demand_constraint(x)

(149.9999999999895, 1.0480505352461478e-12, 0.0)

In [ ]:
result.message, result.status, result.success, result.nfev, result.nit

('Optimization terminated successfully', 0, True, 8, 2)

Note the constraints can be slightly negative, this is expected with numerical solvers

## Are there better solutions?

In [ ]:
x = [10, 0, 0]
-objective(x), capacity_constraint(x), demand_constraint(x)

(100, 5, 0)

In [ ]:
x = [8, 1, 1]
-objective(x), capacity_constraint(x), demand_constraint(x)

(130, 2, 0)

In [ ]:
x = [7, 1, 2]
-objective(x), capacity_constraint(x), demand_constraint(x)

(150, 0, 0)

In [ ]:
x = [6, 3, 1]
-objective(x), capacity_constraint(x), demand_constraint(x)

(150, 0, 0)

In [ ]:
x = [6, 2, 2]
-objective(x), capacity_constraint(x), demand_constraint(x)

(160, -1, 0)

In [ ]:
x0 = [6, 3, 1]
result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)

# Print the results
print('Optimal solution:', result.x)
print('Optimal objective value:', result.fun)
print(result.message, result.status, result.success, result.nfev, result.nit)
x = result.x
-objective(x), capacity_constraint(x), demand_constraint(x)

Optimal solution: [6. 3. 1.]
Optimal objective value: -150.0
Optimization terminated successfully 0 True 4 1


(150.0, 0.0, 0.0)

Try converting the demand equality to an inequality, as the stock can be larger than the demand

In [ ]:
x0 = [6, 3, 1]
cons = ({'type': 'ineq', 'fun': capacity_constraint},
        {'type': 'ineq', 'fun': demand_constraint})

result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)

# Print the results
print('Optimal solution:', result.x)
print('Optimal objective value:', result.fun)
print(result.message, result.status, result.success, result.nfev, result.nit)

x = result.x
objective(x), capacity_constraint(x), demand_constraint(x)

Optimal solution: [6. 3. 1.]
Optimal objective value: -150.0
Optimization terminated successfully 0 True 4 1


(-150.0, 0.0, 0.0)

In [ ]:
x0 = [7, 1, 2]
cons = ({'type': 'ineq', 'fun': capacity_constraint},
        {'type': 'ineq', 'fun': demand_constraint})

result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)

# Print the results
print('Optimal solution:', result.x)
print('Optimal objective value:', result.fun)
print(result.message, result.status, result.success, result.nfev, result.nit)

x = result.x
-objective(x), capacity_constraint(x), demand_constraint(x)

Optimal solution: [7. 1. 2.]
Optimal objective value: -150.0
Optimization terminated successfully 0 True 4 1


(-150.0, 0.0, 0.0)

# COBYLA (Constrained Optimization BY Linear Approximation )


Note cobyla does not accept equality constraint, neither bounds. So we need to convert demand equality to an inequality, and rewrite bounds as contraints (x[0] > 0, for instance)

In [ ]:
x0 = [1, 1, 1]
cobyla_cons = ({'type': 'ineq', 'fun': capacity_constraint},
               {'type': 'ineq', 'fun': demand_constraint},
               {'type': 'ineq', 'fun': lambda x: x[0]},
               {'type': 'ineq', 'fun': lambda x: x[1]},
               {'type': 'ineq', 'fun': lambda x: x[2]})

result = minimize(objective, x0, method='cobyla', constraints=cobyla_cons)
print('Optimal solution:', result.x)
print('Optimal objective value:', result.fun)
print(result.message, result.status, result.success, result.nfev)

x = result.x
-objective(x), capacity_constraint(x), demand_constraint(x)

Optimal solution: [5.61158858 3.77682283 0.61158858]
Optimal objective value: -150.00000000000003
Optimization terminated successfully. 1 True 42


(-150.00000000000003, 8.881784197001252e-16, -3.552713678800501e-15)

In [ ]:
x0 = [6, 3, 1]

result = minimize(objective, x0, method='cobyla', constraints=cobyla_cons)
print('Optimal solution:', result.x)
print('Optimal objective value:', result.fun)

x = result.x
objective(x), capacity_constraint(x), demand_constraint(x)

Optimal solution: [6.00004082 2.99991835 1.00004082]
Optimal objective value: -150.0


(-150.0, 1.7763568394002505e-15, 0.0)